# Classical Mechanics I (PHYS 311)
## Studio 9

*Name: Evan Toon*

*Date: 10-10-2025*

## More complex animations

Make sure you were able to do everything in Studio 8!

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Remember we have these magic lines to give us access to the animation and conversion functionalities.

In [2]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display

Last time, we defined a function `plotSomeStuff`.

```python
def plotSomeStuff(x_array, y_array, nframes=100, ax = None, fig = None):
    if ax == None or fig == None:
        fig, ax = plt.subplots()
        ax.set_xlim(-10,10)
        ax.set_ylim(-10,10)

    phi = linspace(0,2*np.pi,100)

    marker1, = ax.plot([],"o") # Let's create an empty plot that has a circle marker
    orbit1, = ax.plot([]) # Let's create an empty plot to eventually become an orbit line

    def animate(frame_num):
        """
        """
        x = x_array[frame_num]
        y = y_array[frame_num]
        
        # Let's put a marker at the last position of these arrays.
        # We're using set_data which is updating the data stored in the marker1 object.
        marker1.set_data([x,y])
        orbit1.set_data([ x_array[:frame_num],y_array[:frame_num] ])
        return

    anim = FuncAnimation(fig, animate, frames=nframes, interval=20)
    display(HTML(anim.to_jshtml()))
    plt.close()

```


Let's now write a new function based on this. Instead of the first two arguments being numpy arrays, let's make them lists of numpy arrays. So instead of the first argument being `x_array` of type `np.array`, let's have it be `x_arrays` which looks like `[np.array, np.array, ...]`. And then have it animate the trajectories of all of the pairs of arrays handed to the function.

In [3]:
def plotSomeStuff(x_arrays, y_arrays, nframes=100, ax = None, fig = None):
    if ax == None or fig == None:
        fig, ax = plt.subplots()
        ax.set_xlim(-10,10)
        ax.set_ylim(-10,10)


    marker = {}
    orbit = {}

    for ipath,(x_array, y_array) in enumerate(zip(x_arrays,y_arrays) ):
        # !!!!!!!!!!!!!!!!!!!!!!!!!!!
        # This is an example of how to loop over all pairs of x and y arrays.
        # The zip function takes two lists and combines them into a single list like:
        # zip( [a, b], [c, d] ) will give [ [a,c], [b,d] ]
        # The enumerate function gives you an index for each object in an iterable list.
        # So enumerate([a,b]) will give a new list [ [0,a], [1,b] ]
        marker[ipath], = ax.plot([],"o")
        orbit[ipath], = ax.plot([])

    def animate(frame_num):
        """
        """
        # !!!!!!!!!!!!!!!!!!!!!!!!!!!
        # Now write some code here that would loop over all pairs of the x and y arrays (see above)
        # and then use the set_data function like last week to edit each object in marker and orbit.
        for ipath,(x_array, y_array) in enumerate(zip(x_arrays,y_arrays) ):
          x = x_array[frame_num]
          y = y_array[frame_num]
          marker[ipath].set_data([x], [y])
          orbit[ipath].set_data(x_array[:frame_num], y_array[:frame_num])
        return list(marker.values()) + list(orbit.values())

    anim = FuncAnimation(fig, animate, frames=nframes, interval=20)
    display(HTML(anim.to_jshtml()))
    plt.close()


# !!!!!!!!!!!!!!!!!!!!
# Construct the x and y arrays for two particle circular orbits (of differing radius)
# going in opposite directions
# Store them as x_array1, y_array1 (to describe the path of particle 1)
#           and x_array2, y_array2 (to describe particle 2's path)
t = linspace(0,2*pi, 100)
r1 = 5
r2 = 3

x_array1 = r1*cos(t)
y_array1 = r1*sin(t)

x_array2 = r2*sin(t)
y_array2 = r2*cos(t)

# Now we can run the function
plotSomeStuff([x_array1,x_array2],[y_array1,y_array2],nframes=len(t))

Output hidden; open in https://colab.research.google.com to view.

## Two-body problem

Now we have a way of plotting the trajectories of multiple particles. Last week, we solved a system of ODEs to describe the motion of a single particle. Recall that we could map the 2-body gravitational problem onto a single particle of "reduced" mass $\mu$ in a gravitational well centered at $\vec{r}=0$. So let's remember how to get back to two particles from this simplified 1D problem.

$$\mu=\frac{m_1m_2}{M}$$

$$\vec{r}_1 = \vec{R}+\frac{m_2}{M}\vec{r}$$

$$\vec{r}_2 = \vec{R}-\frac{m_1}{M}\vec{r}$$

Let's set $\vec{R}$ to 0 such that we're in the system's CoM frame at all times. This gives us all the ingredients we need.

Solve the same ODE as last week, which will give you the x and y components of the vector $\vec{r}$ as a function of time. Then convert those solutions into two separate vectors $\vec{r}_1$ and $\vec{r}_2$. Then hand those to your multi-particle plotting script above. You'll need to define values for the masses of your two particles, then calculate $\mu$, and make sure you hand it to your ode_system call (using the `args=(thing,)` in your `odeint` call).

Let's start with $m_1=m_2$.

In [4]:
from scipy.integrate import odeint

In [5]:

def ode_system(inputs,t,m):
    """
    This function represents a series of first order ODEs.

    Return: List of expressions for the first time derivative of the inputs, in order.
    """

    # Parse the inputs list to positions x,y and vector magnitudes xdot,ydot
    [x,y,xdot,ydot] = inputs

    r = np.hypot(x,y) # This is a handy function that computes sqrt(x^2+y^2) for you
    phi = np.arctan2(y,x) # And this calculates the polar angle of our vector

    Fr = -1/(r*r)
    Fx = Fr*np.cos(phi)
    Fy = Fr*np.sin(phi)

    # Return the expressions for the time-derivatives of the inputs.
    return [xdot,ydot,Fx/m,Fy/m]


# solve the ODE for position r of mass mu.
# Convert the result in terms of r1 and r2. Plot these particle paths.

G = 1.0
m1 = 1.0
m2 = 1.0
M = m1 + m2
mu = (m1 * m2) / M

r0 = 5.0
v0 = sqrt(G * M / r0)

x0, y0 = r0, 0.0
xdot0, ydot0 = 0.0, v0
inputs0 = [x0, y0, xdot0, ydot0]

t = np.linspace(0, 50, 250)

sol = odeint(ode_system, inputs0, t, args=(mu,))
x = sol[:, 0]
y = sol[:, 1]

r1x = (m2 / M) * x
r1y = (m2 / M) * y
r2x = -(m1 / M) * x
r2y = -(m1 / M) * y

plotSomeStuff([r1x, r2x], [r1y, r2y], nframes=len(t))

Output hidden; open in https://colab.research.google.com to view.

Play with your initial conditions to find some bound and unbound 2-body orbits. If you think you have an unbound orbit, make sure you let your time array go to large enough values to make sure they don't later come back!

In [6]:
G = 1.0
m1 = 1.0
m2 = 1.0
M = m1 + m2
mu = (m1 * m2) / M

r0 = 5.0

# Double initial velocity
v0 = 2* sqrt(G * M / r0)

x0, y0 = r0, 0.0
xdot0, ydot0 = 0.0, v0
inputs0 = [x0, y0, xdot0, ydot0]

t = np.linspace(0, 50, 250)

sol = odeint(ode_system, inputs0, t, args=(mu,))
x = sol[:, 0]
y = sol[:, 1]

r1x = (m2 / M) * x
r1y = (m2 / M) * y
r2x = -(m1 / M) * x
r2y = -(m1 / M) * y

plotSomeStuff([r1x, r2x], [r1y, r2y], nframes=len(t))

Output hidden; open in https://colab.research.google.com to view.

Now start to play with the masses to see how the situation changes! The orbits change significantly!

In [7]:
G = 1.0

# Decreased mass by 0.5
m1 = 0.5

# Increased mass by 1.5
m2 = 2.5

M = m1 + m2
mu = (m1 * m2) / M

r0 = 5.0
v0 = sqrt(G * M / r0)

x0, y0 = r0, 0.0
xdot0, ydot0 = 0.0, v0
inputs0 = [x0, y0, xdot0, ydot0]

t = np.linspace(0, 100, 250)

sol = odeint(ode_system, inputs0, t, args=(mu,))
x = sol[:, 0]
y = sol[:, 1]

r1x = (m2 / M) * x
r1y = (m2 / M) * y
r2x = -(m1 / M) * x
r2y = -(m1 / M) * y

plotSomeStuff([r1x, r2x], [r1y, r2y], nframes=len(t))

Output hidden; open in https://colab.research.google.com to view.

**EC(+0.5): Define a new plotting function which can take the marker sizes as arguments, so that each circle size can be controlled individually. Then plot a 2-body problem orbit with two different mass values, and have the area of the circle be proportional to the mass of the particle (remember that the `plot` marker size changes the diameter of the circle).**

And that's it! You've solved the 2-body problem!